# Azure AI Agent Service နှင့် Semantic Kernel ကို အသုံးပြု၍ RAG

ဒီကုဒ်အပိုင်းက `Azure AI Agent Service` နဲ့ `Semantic Kernel` ကို အသုံးပြုပြီး retrieval-augmented generation (RAG) အတွက် Azure AI agent တစ်ခုကို ဖန်တီးပြီး စီမံခန့်ခွဲပုံကို ပြသထားပါတယ်။ Agent က အသုံးပြုသူရဲ့ မေးခွန်းတွေကို ရှာဖွေထားတဲ့ အကြောင်းအရာအခြေခံပြီး ပြန်လည်ဖြေကြားပေးပါတယ်။


SQLite ဗားရှင်းပြင်ဆင်မှု  
သင်အောက်ပါအမှားကိုတွေ့ရှိပါက -  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

သင်၏ notebook အစမှာ ဒီ code block ကို uncomment လုပ်ပါ:  


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### ပက်ကေ့ဂျ်များကို သွင်းယူခြင်း  
အောက်ပါကုဒ်သည် လိုအပ်သော ပက်ကေ့ဂျ်များကို သွင်းယူပါသည်။  


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# Semantic Kernel နှင့် Azure AI Agent Service ဖြင့် Retrieval-Augmented Generation

ဒီနမူနာက **Azure AI Agent Service** ကိုအသုံးပြုပြီး **Retrieval-Augmented Generation (RAG)** ကို ဘယ်လိုလုပ်ဆောင်ရမယ်ဆိုတာကို ပြသပေးပါတယ်။ ဒါကို ဘာသာစကားမော်ဒယ်နဲ့ တစ်ဦးချင်းအထူးနယ်ပယ်ဆိုင်ရာ အကြောင်းအရာ (တင်သွင်းထားတဲ့စာရွက်စာတမ်း) တွဲဖက်ပြီး လုပ်ဆောင်ပါတယ်။

### ဘယ်လိုအလုပ်လုပ်သလဲ

1. **စာရွက်စာတမ်းတင်သွင်းခြင်း**: Markdown ဖိုင်တစ်ခု (document.md) ကို Agent Service သို့ တင်သွင်းပြီး အချက်အလက် (Contoso ရဲ့ ခရီးသွားအာမခံမူဝါဒ) ပါဝင်ပါတယ်။

2. **Vector Store ဖန်တီးခြင်း**: စာရွက်စာတမ်းကို vector store ထဲမှာ အညွှန်းပြုလုပ်ပြီး အကြောင်းအရာအပေါ် semantic search လုပ်နိုင်အောင် ပြင်ဆင်ထားပါတယ်။

3. **Agent ကိုဖွဲ့စည်းခြင်း**: `gpt-4o` မော်ဒယ်ကို အသုံးပြုပြီး အောက်ပါ တိကျတဲ့ညွှန်ကြားချက်များနဲ့ Agent ကို ဖွဲ့စည်းထားပါတယ်။
   - ရှာဖွေထားတဲ့စာရွက်စာတမ်းအကြောင်းအရာအပေါ် အခြေခံပြီးသာ မေးခွန်းများကို ဖြေဆိုရမည်။
   - မေးခွန်းက အကျုံးအဝင်အပြင်မှာရှိရင် ဖြေဆိုရန် ငြင်းဆိုရမည်။

4. **File Search Tool ပေါင်းစည်းခြင်း**: `FileSearchTool` ကို Agent နဲ့ ပေါင်းစည်းထားပြီး မော်ဒယ်က စာရွက်စာတမ်းထဲက အညွှန်းထားတဲ့ အကြောင်းအရာများကို ရှာဖွေပြီး inference အတွင်းမှာ ပြန်လည်ရယူနိုင်အောင် ပြုလုပ်ထားပါတယ်။

5. **အသုံးပြုသူနှင့် အပြန်အလှန်ဆက်သွယ်မှု**: အသုံးပြုသူများက မေးခွန်းများမေးနိုင်ပါတယ်။ စာရွက်စာတမ်းထဲမှာ သက်ဆိုင်ရာ အချက်အလက်တွေ ရှိနေခဲ့ရင် Agent က အခြေခံထားတဲ့ အဖြေကို ဖန်တီးပေးပါမယ်။  
   မရှိခဲ့ရင်တော့ စာရွက်စာတမ်းထဲမှာ လုံလောက်တဲ့ အချက်အလက် မရှိကြောင်း Agent က တိတိကျကျ ပြောပြပါမယ်။


### အဓိက Function



အရင်ဆုံး `az login` ကို Azure CLI ကို အသုံးပြု၍ လုပ်ဆောင်ပါ၊ သင့်အနေဖြင့် `DefaultAzureCredential` ကို အသုံးပြုစဉ် မှန်ကန်သော authentication context ကို ပေးရန် လိုအပ်ပါသည်။ Azure AI Agent Service သည် API keys မသုံးပါ။


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

ကျေးဇူးပြု၍ Markdown ဖိုင်ကို ပေးပါ၊ ပြန်လည်ဘာသာပြန်ပေးပါမည်။



---

**အကြောင်းကြားချက်**:  
ဤစာရွက်စာတမ်းကို AI ဘာသာပြန်ဝန်ဆောင်မှု [Co-op Translator](https://github.com/Azure/co-op-translator) ကို အသုံးပြု၍ ဘာသာပြန်ထားပါသည်။ ကျွန်ုပ်တို့သည် တိကျမှုအတွက် ကြိုးစားနေသော်လည်း၊ အလိုအလျောက် ဘာသာပြန်မှုများတွင် အမှားများ သို့မဟုတ် မတိကျမှုများ ပါဝင်နိုင်သည်ကို သတိပြုပါ။ မူရင်းဘာသာစကားဖြင့် ရေးသားထားသော စာရွက်စာတမ်းကို အာဏာရှိသော ရင်းမြစ်အဖြစ် သတ်မှတ်သင့်ပါသည်။ အရေးကြီးသော အချက်အလက်များအတွက် လူ့ဘာသာပြန်ပညာရှင်များမှ ပရော်ဖက်ရှင်နယ် ဘာသာပြန်မှုကို အကြံပြုပါသည်။ ဤဘာသာပြန်မှုကို အသုံးပြုခြင်းမှ ဖြစ်ပေါ်လာသော အလွဲအလွဲအချော်များ သို့မဟုတ် အနားလွဲမှုများအတွက် ကျွန်ုပ်တို့သည် တာဝန်မယူပါ။
